In [1]:
import numpy as np
import time

In [2]:
# -------------------- Parameters --------------------
D = 30                      # Dimensions
NP = 100                     # Population size
F = 0.5                     # Scaling factor
CR = 0.9                    # Crossover probability
lower_bound = -5.0
upper_bound = 10.0
max_generations = 2600
tol = 1e-5

In [3]:
# -------------------- Zakharov Function --------------------
def zakharov(x):
    x = np.asarray(x)
    i = np.arange(1, len(x)+1)
    term1 = np.sum(x**2)
    term2 = (np.sum(0.5 * i * x))**2
    term3 = (np.sum(0.5 * i * x))**4
    return term1 + term2 + term3

In [4]:
# -------------------- Initialization --------------------
np.random.seed(42)
pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([zakharov(ind) for ind in pop])
best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]

In [5]:
# -------------------- DE Main Loop --------------------
generation = 0
fitness_history = [best_fitness]
start_time = time.time()

while generation < max_generations and best_fitness > tol:
    generation += 1

    for i in range(NP):
        # Mutation
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, size=3, replace=False)
        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = np.clip(donor, lower_bound, upper_bound)

        # Crossover
        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            trial[j] = donor[j] if np.random.rand() < CR or j == j_rand else pop[i][j]

        # Selection
        f_trial = zakharov(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()

    fitness_history.append(best_fitness)

    if generation % 200 == 0 or best_fitness <= tol:
        print(f"Generation {generation:3d}: Best Fitness = {best_fitness:.6e}")

    if best_fitness <= tol:
        print(f"\nTerminated at generation {generation} (fitness ≤ {tol}).")
        break


Generation 200: Best Fitness = 1.818452e+02


Generation 400: Best Fitness = 1.169846e+02


Generation 600: Best Fitness = 3.886132e+01


Generation 800: Best Fitness = 2.031466e+01


Generation 1000: Best Fitness = 5.407582e+00


...

Generation 1800: Best Fitness = 5.203212e-03


Generation 2000: Best Fitness = 7.517084e-04


Generation 2200: Best Fitness = 1.748246e-04


Generation 2400: Best Fitness = 3.446088e-05


Generation 2564: Best Fitness = 9.475435e-06

Terminated at generation 2564 (fitness ≤ 1e-05).


In [6]:
# -------------------- Final Output --------------------
end_time = time.time()
print("\nFinal Result:")
print(f"Best Vector: {best_vector}")
print(f"Best Fitness: {best_fitness}")
print(f"Total Time Taken: {end_time - start_time:.4f} seconds")


Final Result:
Best Vector: [-3.10609726e-04 -3.72487311e-04 -5.97268879e-04  1.56570816e-04
 -1.69664802e-04  4.70942652e-06  3.01552676e-04  1.61439445e-05
 -1.36799999e-04  5.62636080e-04 -7.41055377e-04  1.15236143e-04
 -7.23836834e-04 -6.24773711e-04  3.80040293e-04 -1.55306287e-04
  5.67305570e-04 -1.30430301e-03 -1.06528227e-03  5.21020596e-04
  1.88400454e-04  1.49143469e-03 -1.96636211e-04 -1.38304241e-04
  5.90594677e-04  1.16391567e-04 -4.90763646e-05  6.14884038e-04
 -2.91758578e-04 -4.02944992e-04]
Best Fitness: 9.475434821962506e-06
Total Time Taken: 32.7593 seconds


In [7]:
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'colab'  # or 'colab' or 'browser' depending on your environment

# Plot Fitness History with Logarithmic Y-Axis
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fitness_history,
    mode='lines+markers',
    line=dict(color='royalblue'),
    marker=dict(size=4),
    name='Best Fitness'
))

fig.update_layout(
    title='DE Convergence on Zakharov Function (Log Scale)',
    xaxis_title='Generation',
    yaxis_title='Best Fitness (log scale)',
    template='plotly_white',
    hovermode='x unified',
    yaxis_type='log'  # 👈 log scale here
)

fig.show()
